In [1]:
import pdb
import os
import numpy as np
import pandas as pd
import pylab
import pylab as plt
from utils import find_nearest
from utils import find_nearest_index

%matplotlib inline

In [2]:
year = 2019
model = 'Steamer'
model = 'ZiPS'
path_data = '/data/baseball/Fangraphs/projections/'

In [3]:
class Projection:
    
    def __init__(self, model = 'ZiPS', year = 2019, path_data = '/data/baseball/Fangraphs/projections/'):
        self.statline = {}
        self.hitters_rank = {}
        self.pitchers_rank= {}
        self.hitters_stats = pd.DataFrame()

        # Read in Batters by Position for Year and Position
        for file in os.listdir(path_data+str(year)+'/'):
            if file.startswith(model) & ~file.endswith('Hitters.csv') & ~file.endswith('Pitchers.csv') :
                #print(os.path.join(path_data+str(year)+'/', file))
                df = pd.read_csv(os.path.join(path_data+str(year)+'/', file))
                fn = str.split(file,'.')[0][-2:]
                if fn == '_C': fn = 'C'
                #print fn
                df['Position'] = fn
                df['Rank'] = 999
                if self.hitters_stats.empty:
                    self.hitters_stats = df
                else:
                    self.hitters_stats = pd.concat([self.hitters_stats, df])
                
                #Read in Rotographs Ranking Predictions
                if fn != 'DH':
                    kk = 1
                    rotographs_file = fn+str('-Table 1.csv')
                    self.hitters_rank[fn] = pd.read_csv(os.path.join(path_data+str(year)+'/RotoGraphsPositionalRankings/', rotographs_file))
                    #self.hitters_rank[fn]['Drafted'] = 0
                    for plr in self.hitters_rank[fn]['PLAYER']:
                        ind = self.hitters_stats['Name'] == plr
                        self.hitters_stats['Rank'][ind] = kk
                        kk += 1
                    
        self.hitters_stats['1B'] = self.hitters_stats['H'] - self.hitters_stats['2B'] - self.hitters_stats['3B'] - self.hitters_stats['HR']
        
        # Read in Pitchers for Year and Position
        for file in os.listdir(path_data+str(year)+'/'):
            if file.startswith(model) & file.endswith('Pitchers.csv'):
                #print(os.path.join(path_data+str(year)+'/', file))
                self.pitchers_stats = pd.read_csv(os.path.join(path_data+str(year)+'/', file))
                self.pitchers_stats['Position'] = 'P'
                self.pitchers_stats['Rank'] = 999
        self.pitchers_stats['CG']  = 0
        self.pitchers_stats['SHO']  = 0
        self.pitchers_stats['SV']  = 0
        self.pitchers_stats['BSV']  = 0

        #Read in Rotographs Ranking Predictions        
        pitcher_positions = ['SP','RP']
        for fn in pitcher_positions:
            k = 1
            rotographs_file = fn+str('-Table 1.csv')
            self.pitchers_rank[fn] = pd.read_csv(os.path.join(path_data+str(year)+'/RotoGraphsPositionalRankings/', rotographs_file))
            #self.pitchers_rank[fn]['Drafted'] = 0
            for plr in self.pitchers_rank[fn]['PLAYER']:
                ind = self.pitchers_stats['Name'] == plr
                self.pitchers_stats['Position'][ind] = fn
                self.pitchers_stats['Rank'][ind] = k
                k += 1
                if fn == 'SP':
                    self.pitchers_stats['CG'][ind] = np.floor(self.pitchers_stats['IP'][ind] * 0.01 * (1./self.pitchers_stats['WHIP'][ind]))
                    self.pitchers_stats['SHO'][ind] = np.ceil(self.pitchers_stats['CG'][ind] *0.55)
                if fn == 'RP':
                    self.pitchers_stats['SV'][ind] = np.floor(self.pitchers_stats['IP'][ind] * 0.5 * (1./self.pitchers_stats['WHIP'][ind]))
                    self.pitchers_stats['BSV'][ind] = np.floor(self.pitchers_stats['IP'][ind] * 0.05 * (1./self.pitchers_stats['WHIP'][ind]))

    def precompute_statlines(self, 
                             batter_categories  = ['R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'], 
                             pitcher_categories = ['W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV' ]):
        b_col_names = batter_categories
        if 'AB' not in b_col_names:
            b_col_names.extend(['AB'])
        if 'Position' not in b_col_names:
            b_col_names.extend(['Position'])    
        if 'Rank' not in b_col_names:
            b_col_names.extend(['Rank']) 
        if 'playerid' not in b_col_names:
            b_col_names.extend(['playerid'])
        if 'Name' not in b_col_names:
            b_col_names.extend(['Name'])     
        self.statline['batters'] = self.hitters_stats[b_col_names]#.groupby('Name').mean()
        self.statline['batters']['Drafted'] = 'False'
        
        p_col_names = pitcher_categories
        if 'IP' not in p_col_names:
            p_col_names.extend(['IP'])
        if 'Position' not in p_col_names:
            p_col_names.extend(['Position'])   
        if 'Rank' not in p_col_names:
            p_col_names.extend(['Rank'])  
        if 'playerid' not in p_col_names:
            p_col_names.extend(['playerid'])
        if 'Name' not in p_col_names:
            p_col_names.extend(['Name'])
        self.statline['pitchers'] = self.pitchers_stats[p_col_names]#.groupby('Name').mean()
        self.statline['pitchers']['Drafted'] = 'False'

In [5]:
ZiPS = Projection()

IOError: File /data/baseball/Fangraphs/projections/2019/RotoGraphsPositionalRankings/1B-Table 1.csv does not exist

In [21]:
ZiPS.precompute_statlines()

/Users/marcoviero/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/marcoviero/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
ZiPS.statline['batters'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,playerid,Name,Drafted
0,76,79,32,4,25,84,5,38,0.276,0.838,507,C,2,11739,J.T. Realmuto,False
1,55,53,19,1,23,69,1,68,0.238,0.811,403,C,4,11368,Yasmani Grandal,False
2,68,53,21,0,28,85,3,44,0.246,0.822,415,C,1,11442,Gary Sanchez,False
3,53,89,24,1,8,53,4,48,0.288,0.770,423,C,5,9166,Buster Posey,False
4,56,75,26,3,14,64,5,51,0.255,0.754,463,C,6,11609,Willson Contreras,False


In [50]:
class Draft:
    
    def __init__(self, projection, number_teams = 12,
                 batters  = {'C','1B','2B', '3B','SS','OF','UTIL'},
                 pitchers = {'SP','RP','P'},
                 number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}, 
                 batter_statline  = {'R':0,'1B':0,'2B':0, '3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}, 
                 pitcher_statline = {'W':0, 'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'SO':0,'ERA':0,'WHIP':0,'BSV':0} ):
        
        self.projection = projection
        self.batters  = batters
        self.pitchers = pitchers
        self.open_positions = number_positions
        self.batter_statline = batter_statline
        self.pitcher_statline = pitcher_statline        
        self.number_teams = number_teams

        bat_rows = []
        bat_cols = ['Name']
        for ibat in self.batters:
            for i in range(self.open_positions[ibat]):
                bat_rows.append(ibat)
        for ibat in self.batter_statline:
            bat_cols.append(ibat)
        print(bat_rows)        
        print(bat_cols)
        
        bat_df = pd.DataFrame(np.zeros([len(bat_rows),len(bat_cols)]), columns = bat_cols)
        bat_df['Position'] = bat_rows
        
        pit_rows = ['Name']
        pit_cols = []
        for ipit in self.pitchers:
            for i in range(self.open_positions[ipit]):
                pit_rows.append(ipit)
        for ipit in self.pitcher_statline:
            pit_cols.append(ipit)
            
        print(pit_rows)        
        print(pit_cols)
        
        pit_df = pd.DataFrame(np.zeros([len(pit_rows),len(pit_cols)]), columns = pit_cols)
        pit_df['Position'] = pit_rows   
        
        self.bat_df = bat_df
        self.pit_df = pit_df
        
        #return bat_df, pit_df
    
    def draft_team(self, draft_position = 'Best'):
        print(self.number_teams)
        self.draftees  = pd.DataFrame()
        for psn in self.batters:
            print(psn)
            if psn != 'UTIL':
                #print(psn)
                ranked_position_df = self.projection.statline['batters'][(self.projection.statline['batters']['Position']==psn) & (self.projection.statline['batters']['Drafted']==False)].sort_values('Rank')[0:self.number_teams]
                rank_stat = np.zeros([len(self.batter_statline),self.number_teams])

                for i in range(len(self.batter_statline)):
                    cat = self.batter_statline.keys()[i]
                    #print(cat)
                    ind = np.argsort(ranked_position_df[cat])
                    if cat in ['HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']:
                        rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                position_rank = np.sum(rank_stat.T, axis=1)
                print(position_rank)

                #for i in (1 + np.arange(self.number_teams - 1)):
                #    print(-i)
                #    print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-i]], np.sort(position_rank)[-i])
                print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]])
                if self.draftees.empty:
                    self.draftees = df.T
                else:
                    self.draftees = pd.concat([self.draftees, df.T])
                
        for psn in self.pitchers:
            if psn != 'P':
                ranked_position_df = self.projection.statline['pitchers'][(self.projection.statline['pitchers']['Position']==psn) & (self.projection.statline['pitchers']['Drafted']==False)].sort_values('Rank')[0:self.number_teams]
                rank_stat = np.zeros([len(self.pitcher_statline),self.number_teams])

                for i in range(len(self.pitcher_statline)):
                    cat = self.pitcher_statline.keys()[i]
                    #print(cat)
                    ind = np.argsort(ranked_position_df[cat])
                    if cat in ['SO', 'W', 'SV', 'ERA', 'CG', 'SHO']:
                        rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
                    else:
                        rank_stat[i, ind]= np.arange(len(ind),0,-1)
                position_rank = np.sum(rank_stat.T, axis=1)
                #for i in (1 + np.arange(self.number_teams - 1)):
                #    print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-i]], np.sort(position_rank)[-i])

                print(psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1])
                
                df = pd.DataFrame([psn,ranked_position_df.Name.values[np.argsort(position_rank)[-1]], np.sort(position_rank)[-1]])
                self.draftees = pd.concat([self.draftees, df.T])
                

In [61]:
ZiPS.statline['batters'].head()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,AB,Position,Rank,playerid,Name,Drafted
0,76,79,32,4,25,84,5,38,0.276,0.838,507,C,2,11739,J.T. Realmuto,False
1,55,53,19,1,23,69,1,68,0.238,0.811,403,C,4,11368,Yasmani Grandal,False
2,68,53,21,0,28,85,3,44,0.246,0.822,415,C,1,11442,Gary Sanchez,False
3,53,89,24,1,8,53,4,48,0.288,0.770,423,C,5,9166,Buster Posey,False
4,56,75,26,3,14,64,5,51,0.255,0.754,463,C,6,11609,Willson Contreras,False


In [62]:
number_teams = 10
psn = 'C'
indt = ZiPS.statline['batters'][(ZiPS.statline['batters']['Position']==psn) & (ZiPS.statline['batters']['Drafted']==False)].sort_values('Rank')[0:number_teams]


In [51]:
tstDft = Draft(ZiPS,number_teams = 12)

['C', 'SS', 'OF', 'OF', 'OF', 'UTIL', '1B', '2B', '3B']
['Name', 'HR', 'R', 'RBI', '2B', 'SB', '1B', '3B', 'BB', 'AVG', 'OPS']
['Name', 'P', 'P', 'P', 'SP', 'SP', 'RP', 'RP']
['SO', 'BB', 'W', 'SV', 'ERA', 'BSV', 'CG', 'L', 'WHIP', 'SHO']


In [52]:
tstDft.draft_team()

12
C
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


IndexError: index 11 is out of bounds for axis 0 with size 0

In [253]:
batter_categories  = ['R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_categories = ['W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV' ]

psn = '3B'
ranked_position_df = bats_list[(bats_list['Position']==psn) & (bats_list['Drafted']==False)].sort_values('Rank')[0:12]
rank_stat = np.zeros([len(batter_categories),len(ind)])

for i in range(len(batter_categories)):
    cat = batter_categories[i]
    #print cat
    ind = np.argsort(ranked_position_df[cat])
    rank_stat[i, ind[::-1]]= np.arange(len(ind),0,-1)
print(psn,np.sum(rank_stat.T, axis=1))

('3B', array([ 91., 110.,  95.,  60.,  42.,  42.,  45.,  57.,  65.,  85.,  39.,
        49.]))


In [276]:
ranked_position_df.Name

1      Nolan Arenado
0       Jose Ramirez
3       Alex Bregman
7     Anthony Rendon
2     Josh Donaldson
12       Travis Shaw
14      Matt Chapman
6     Matt Carpenter
9     Eugenio Suarez
4        Kris Bryant
5      Justin Turner
15     Rafael Devers
Name: Name, dtype: object

In [11]:
number_positions = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}

In [12]:
print(number_positions.keys())
print(number_positions.values())
np.argsort(number_positions.values())[::-1] 

['C', 'BN', 'UTIL', 'P', '1B', '2B', '3B', 'RP', 'SS', 'OF', 'SP']
[1, 5, 1, 3, 1, 1, 1, 2, 1, 3, 2]


array([ 1,  9,  3, 10,  7,  8,  6,  5,  4,  2,  0])

In [13]:
ind = ZiPS.hitters_stats['Name'] == ZiPS.hitters_rank['C']['PLAYER'][0] 
ZiPS.hitters_stats[ind]
#ZiPS.pitchers_stats.head()

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,OPS,wOBA,Fld,BsR,WAR,ADP,playerid,Position,Rank,1B
2,Gary Sanchez,Yankees,110,467,415,102,21,0,28,68,...,0.822,0.35,-5.5,-0.1,3.1,55.3,11442,C,1,53


In [14]:
statline = {}
statline['batter_statline'] = {'R':0,'1B':0,'2B':0,'3B':0,'HR':0,'RBI':0,'SB':0,'BB':0,'AVG':0,'OPS':0}
statline['pitcher_statline'] = {'W':0,'L':0,'CG':0,'SHO':0,'SV':0,'BB':0,'K':0,'ERA':0,'WHIP':0,'BSV':0}

In [15]:
number_positions={'C':1,'1B':1,'2B':1,'3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}